In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:

import argparse
import torch.optim as optim
from torchvision import datasets, transforms

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

# Train this model with 60 epochs and after process every 300 batches log the train status 
args = parser.parse_args(['--epochs', '60', '--log-interval', '300'])

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332586
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.839610
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.714290
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.413216

Test set: Average loss: 0.2055, Accuracy: 9401/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.474389
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.722072
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.405250
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.312333

Test set: Average loss: 0.1317, Accuracy: 9596/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.354408
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.492845
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.256229
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.205076

Test set: Average loss: 0.1025, Accuracy: 9680/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.278850
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.405397
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.353746
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.173339

Test set: Averag

Train Epoch: 31 [57600/60000 (96%)]	Loss: 0.106925

Test set: Average loss: 0.0378, Accuracy: 9888/10000 (99%)

Train Epoch: 32 [0/60000 (0%)]	Loss: 0.131782
Train Epoch: 32 [19200/60000 (32%)]	Loss: 0.070957
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.166688
Train Epoch: 32 [57600/60000 (96%)]	Loss: 0.077503

Test set: Average loss: 0.0324, Accuracy: 9898/10000 (99%)

Train Epoch: 33 [0/60000 (0%)]	Loss: 0.051014
Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.057107
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.017279
Train Epoch: 33 [57600/60000 (96%)]	Loss: 0.052133

Test set: Average loss: 0.0352, Accuracy: 9895/10000 (99%)

Train Epoch: 34 [0/60000 (0%)]	Loss: 0.048239
Train Epoch: 34 [19200/60000 (32%)]	Loss: 0.236167
Train Epoch: 34 [38400/60000 (64%)]	Loss: 0.101280
Train Epoch: 34 [57600/60000 (96%)]	Loss: 0.138531

Test set: Average loss: 0.0328, Accuracy: 9901/10000 (99%)

Train Epoch: 35 [0/60000 (0%)]	Loss: 0.157204
Train Epoch: 35 [19200/60000 (32%)]	Loss: 0.095291
Train Epoch

In [6]:
torch.save(model.state_dict(), 'mnist.pth')

In [10]:
from torch.autograd import Variable

# Load the trained model from file
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))

# Export the trained model to ONNX
dummy_input = Variable(torch.randn(1, 1, 28, 28)) # one black and white 28 x 28 picture will be the input to the model
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")